In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import re
# from transliterate import translit
sess = tf.InteractiveSession()
random.seed(1234)

In [2]:
# translit('Привет!', 'ru', reversed=True)

In [3]:
lines = []
with open('./compound.txt','r', errors='ignore') as f:
    for l in f:
        lines.append(l)

In [4]:
valid = ' !"()-0123456789:;.,?abcdefghijklmnopqrstuvwxyzабвгдежзийклмнопрстуфхцчшщъыьэюяё'
valid = set(valid)
lines = [[it for it in line.lower() if it in valid] for line in lines]
lines = map(lambda x: ''.join(x), lines)

In [5]:
subber = lambda x: re.sub('\s+', ' ', x)

In [6]:
lines = map(subber, lines)
lines = filter(lambda x: len(x) > 10, lines)
lines = map(lambda x: x.strip(' ').strip().lower(), lines)
lines = map(lambda x: x.split('.'), lines)
lines = list(lines)

In [7]:
lines = sum(lines, [])
lines = map(lambda x: x.split(','), lines)
lines = sum(lines, [])
lines = map(lambda x: x.strip(' '), lines)
# lines = map(lambda x: translit(x, 'ru', reversed=True), lines)
lines = list(filter(lambda x: len(x) > 10, lines))

In [8]:
lines[400]

'давным-давно (так давно'

In [9]:
# names = pd.read_csv('./all.csv')['content']
# names = names.apply(lambda x: x.split('\r\n'))

In [10]:
# names = list(names)

In [11]:
# names = sum(names, [])
# names = list(map(lambda x: x.strip(' ').lower(), names))
# names = list(filter(lambda x: len(x) > 0, names))

In [12]:
names = lines

In [13]:
# names = np.loadtxt('/home/nikita/tmp/Practical_RL/week7_[recap]_rnn/names', dtype=str, delimiter=';').tolist()

In [14]:
random.shuffle(names)

In [15]:
subber(names[4])

'что не сможет прийти'

In [16]:
len(names)

120032

In [17]:
chars = sorted(set(''.join(names)+'_'))
dic = {x:i for i,x in enumerate(chars)}

In [18]:
class DistLSTM:
    def __init__(self, dim, name='DistLSTM', reuse=None):
        self.dim = dim
        self.name = name
        self.reuse = reuse
        
        with tf.variable_scope(self.name, reuse=reuse):
            cells = [tf.nn.rnn_cell.LSTMCell(128, 
                                                name='cell_{}'.format(i), 
                                                activation=tf.nn.tanh) for i in range(3)]
            self.cell = tf.nn.rnn_cell.MultiRNNCell(cells)
            self.post_cell = lambda x: self.dense(x, dim, name='d1')
            self.init_dist = tf.get_variable('init_dist',[1,dim], trainable=True,
                                             initializer=tf.random_normal_initializer(stddev=0.01, mean=0.2))
        
    def forward_string_lookup(self, strings, dic):
        inp = strings
        
        forward_lookup = tf.py_func(lambda x: self._convert_to_ix(x, dic), [inp], tf.int64)
        forward_lookup = tf.reshape(forward_lookup, (-1,))
        forward_lookup = tf.nn.embedding_lookup(tf.diag(tf.ones(len(chars))), forward_lookup)
        forward_lookup = tf.reshape(forward_lookup, [tf.shape(inp)[0], -1, len(chars)])
        forward_lookup = tf.cast(forward_lookup, tf.float32)
        return forward_lookup
    
    def dense(self, inp, dim, name='dense'):
        with tf.variable_scope(name, initializer=tf.random_normal_initializer(stddev=0.01)):
            W = tf.get_variable('W', [inp.shape[-1], dim])
            b = tf.get_variable('b', [1, dim])
            out = tf.matmul(inp, W) + b
        return out
    
    def logdens(self, seq):
        with tf.variable_scope(self.name, reuse=self.reuse):
            batch_size, s_len = tf.shape(seq)[0], tf.shape(seq)[1]

            cell = self.cell

            s_t = tf.transpose(seq, [1,0,2])
            init_state = cell.zero_state(batch_size=batch_size, dtype=tf.float32)

            init = (tf.zeros([batch_size, cell.state_size[0][0]]), init_state)
            out,_ = tf.scan(lambda prev, x: cell(x, prev[1]), s_t, initializer=init)
            out = tf.transpose(out, [1,0,2])
            
            out_dim = out.shape
            
            out = tf.reshape(out, [-1, out_dim[-1]])
            out = self.post_cell(out)
            out = tf.reshape(out, [batch_size, s_len, self.dim])
            
            preds = out[:,:-1]
            target = seq[:,1:]
                        
            init_logits = tf.tile(self.init_dist, [batch_size,1])
            init_nll = tf.nn.softmax_cross_entropy_with_logits_v2(labels=seq[:,0], logits=init_logits)
            init_nll = init_nll[:,tf.newaxis]
            
            nll = tf.nn.softmax_cross_entropy_with_logits_v2(labels=target, logits=preds)
            nll = tf.concat([init_nll, nll], axis=1)
            return -nll
        
    def sample(self):
        with tf.variable_scope(self.name, reuse=self.reuse):
            init_sample = tf.distributions.Multinomial(total_count=1., logits=self.init_dist).sample()
            
            cell = self.cell

            init_state = cell.zero_state(batch_size=1, dtype=tf.float32)

            init = (init_sample, init_state)
            
            def step(prev):
                x = prev[0]
                state = prev[1]
                cell_step = cell(x, state)
                post_step = self.post_cell(cell_step[0])
                post_step = tf.distributions.Multinomial(total_count=1., logits=post_step).sample()
                return post_step, cell_step[1]
            
            out,_ = tf.scan(lambda prev, _: step(prev), tf.range(40), initializer=init)
            out = tf.transpose(out, [1,0,2])
            out = tf.concat([init_sample[:,tf.newaxis,:], out], axis=1)
            return out                
            
    def backward_string_lookup(self, encs, dic):
        encs = tf.cast(encs, tf.bool)
        strs = tf.py_func(lambda x: self._convert_from_enc(x, dic), [encs], tf.string)
        return strs
        
    @staticmethod
    def _convert_to_ix(names, dic):
        if type(names[0]) != str:
            names = list(map(lambda x: x.decode('utf-8'), names))
        chars = []
        max_len = max([len(x) for x in names])
        filler = dic['_']
        for name in names:
            chars.append([])
            for s in name:
                chars[-1].append(dic[s])
            chars[-1] += [filler]*(max_len-len(name))
        return np.array(chars)
    
    @staticmethod
    def _convert_from_enc(encs, dic):
        rev_dic = {i:x for x,i in dic.items()}
        ret = []
        for row in encs:
            table = np.array([range(len(dic))]*len(row))
            ixs = table[row]
            chars = [rev_dic[ix] for ix in ixs]
            string = ''.join(chars)
            ret.append(string)
            print(string)
        return np.array(ret)

In [19]:
len(names)

120032

In [20]:
train = names[:118000]
test = names[118000:]

In [21]:
train_data = tf.Variable(np.array(train), trainable=False, name='train_data')
test_data = tf.Variable(np.array(test), trainable=False, name='test_data')

shuffle_op = tf.assign(train_data, tf.random_shuffle(train_data))
train_crop = tf.random_crop(train_data, [50])
# test_data = tf.random_crop(train_data, [0])

In [22]:
dlstm = DistLSTM(len(chars), reuse=tf.AUTO_REUSE)

In [23]:
fwd_lk = dlstm.forward_string_lookup(train_crop, dic)

fwd_lk_test = dlstm.forward_string_lookup(test_data, dic)

train_loss = -tf.reduce_mean(dlstm.logdens(fwd_lk))

test_loss = -tf.reduce_mean(dlstm.logdens(fwd_lk_test))

In [24]:
sample = dlstm.sample()
reconstr = dlstm.backward_string_lookup(sample, dic)

In [25]:
# !rm -R /tmp/tfdbg

In [26]:
train_sum = tf.summary.scalar('train_loss', train_loss)
tf.summary.scalar('test_loss', test_loss)
summary = tf.summary.merge_all()
!mkdir /tmp/tfdbg
writer = tf.summary.FileWriter('/tmp/tfdbg/1')

mkdir: cannot create directory ‘/tmp/tfdbg’: File exists


In [27]:
opt = tf.train.AdamOptimizer(0.0004).minimize(train_loss)

In [28]:
tf.global_variables_initializer().run()

In [31]:
ss = sample.eval()
dlstm._convert_from_enc(ss.astype('bool'), dic)

но вечером набы увидала__________________


array(['но вечером набы увидала__________________'], dtype='<U41')

In [ ]:
for epoch in range(1000):
    for batch in range(400):
        opt.run()
        if batch % 20 == 0:
            s = train_sum.eval()
            writer.add_summary(s)

    print(train_loss.eval())
    s = summary.eval()
    writer.add_summary(s)
    shuffle_op.eval()
    
    ss = sample.eval()
    dlstm._convert_from_enc(ss.astype('bool'), dic)
#     print(reconstr.eval())

0.7760212
с чгоми хилибаоным_______________________
0.45494968
поднялись к тому_________________________
0.431212
что васово отыплавивший вызяу этого______
0.47667927
но и скледвульно_________________________
0.7776845
что я бы_________________________________
0.46388224
лилиан павложали закорка и соглашившема в
0.6489019
мы в ранецник и потребуться нам востом___
0.25645992
что казалось_____________________________
0.57164174
что протянул рядой_______________________
0.56273395
я ужаснул лицо то________________________
0.43044356
как только же при избавления_____________
0.49030954
и не помнял крахвирый обраховом сотрестит
0.4945532
кроме сказались и побед__________________
0.40282422
хоть после свяци и была для конца большое
0.45522383
увна осыделся____________________________
0.36085746
что его было я страх добрыцтя____________
0.50631845
и к стуктере был стратают________________
0.50028116
и еще высаковица этого произошло и очень 
0.59362847
что близко ненавистью бигу_______________

In [29]:
saver = tf.train.Saver()

In [30]:
saver.save(sess, '/tmp/save')

INFO:tensorflow:Restoring parameters from /tmp/save
